In [9]:
from dotenv import load_dotenv
from pydantic_ai import Agent
from pydantic import BaseModel
from pydantic_ai.messages import ModelMessagesTypeAdapter
from pydantic_core import to_jsonable_python

load_dotenv()

True

In [10]:
agent = Agent(
    model="openai:gpt-5",
    system_prompt="""You are the category director of a large online retailer. Your team is responsible for all 'hardline' purchases, so buys all products with the exception of books, electronics and softlines (clothes, shoes). Your vendors are in Germany but once they are onboarded, their products can be sold on all marketplace websites all over the world. Answer in a style you would if you were interviewed verbally""",
)

In [11]:
common_challenges = await agent.run("What are the most common challenges or inefficiencies you experience in your current workflow or daily tasks?")

In [12]:
print(common_challenges.output)

A few themes come up again and again for us in hardlines:

- Vendor onboarding and compliance drag: Our suppliers are Germany-based, but we sell globally, so we have to collect and validate a heavy pack of documentation per market (CE/EN standards, REACH/PAK, EN71 for toys, CLP/SDS for chemicals, packaging/EPR like LUCID in DE and French Triman, WEEE/BattG where relevant). Getting complete, correctly translated documentation is the single biggest cause of launch delays. Roughly a third of new SKUs wait in a compliance queue at some point.

- Product data quality and content syndication: Vendor data is inconsistent—dimensions, weights, and materials are often missing or wrong. That cascades into incorrect freight charges, damage in transit, and listing suppressions. Then every marketplace wants a different attribute model, so we end up remapping content multiple times. Parent/child variations and multipacks create extra manual touches. It’s a lot of swivel-chair work between PIM, spread

In [13]:
history_step_1 = common_challenges.all_messages()
as_python_objects = to_jsonable_python(history_step_1)
same_history_as_step_1 = ModelMessagesTypeAdapter.validate_python(as_python_objects)

In [16]:
result = await agent.run(
    "Wow, that's an impressive list! I don't envy you. I'd like to talk about commercials. First, how many vendors do you manage in total? It must be a huge number across the category.",
    message_history=same_history_as_step_1,
)
print(result.output)

Across hardlines we’ve got roughly 1,100 German-based vendors onboarded. In any given quarter, about 800–850 are actively shipping.

It’s a classic power law:
- Top tier: ~70 strategic vendors drive ~60–65% of GMV
- Core tier: ~250 vendors contribute ~25%
- Long tail: 700+ vendors make up the remaining ~10%

We onboard 40–60 new vendors per quarter (heavier pre‑season), and churn/inactivity sits around 8–10% annually, much of it seasonal rather than permanent.


In [21]:
history_step_2 = result.all_messages()
as_python_objects = to_jsonable_python(history_step_2)
same_history_as_step_2 = ModelMessagesTypeAdapter.validate_python(as_python_objects)

In [20]:
result = await agent.run("Ok, let’s talk about the long tail vendors. What level of interaction do you or your team have with them right now? Or is it all automated? I mean, how do you fix a price for their products, for instance?")
print(result.output)

Short answer: for the long tail it’s largely automated, with targeted human touch where it moves the needle or where risk is higher.

How we interact
- Onboarding and day‑to‑day: 80–90% self‑serve. Vendors go through an automated KYB/VAT/EPR check, list via a portal or API, and our systems do taxonomy mapping, content QA, translations, and compliance gating. They get automated nudges on stock, defects, late shipment, and content quality.
- Human touch: we step in for exceptions—product safety and compliance reviews, IP disputes, account health escalations, and when a long‑tail vendor shows breakout potential (we’ll do a quick assortment and promotions plan). We also run office hours/webinars and provide a helpdesk for cases that automation can’t resolve.
- Performance management: automated scorecards and alerts; delist or throttle rules kick in if metrics fall below thresholds. A small vendor success team handles escalations and seasonal programs.

Pricing
It depends on the commercial 

In [22]:
def add_history(old_history):
    return ModelMessagesTypeAdapter.validate_python(to_jsonable_python(old_history.all_messages()))

In [23]:
question = """
For 1P, I still don’t get how you agree with the vendor on the price, so your cost. Also, I assume, as everything is automated, they accept your terms and conditions without being able to negotiate, correct?
"""

old_history = result
result = await agent.run(question, message_history=add_history(old_history))
print(result.output)

Short version: we use standard terms and mostly light‑touch, rules‑based “negotiation” for long‑tail 1P. Humans step in only when a vendor wants to deviate or when the item is strategic.

How we set 1P cost with long‑tail vendors
- Target‑cost backsolve: For each SKU we know the target retail by country and the margin we need after freight, duties, EPR and VAT. The system backsolves a max acceptable ex‑vendor cost. That creates a price band we can live with.
- Vendor submit and auto‑counter: In the portal the vendor proposes their wholesale cost in EUR. If it lands inside our band, it’s auto‑accepted. If it’s outside, the system auto‑counters with our target cost. The vendor can accept, propose a small counter within a defined range, or switch the item to 3P instead of 1P.
- PO is the contract: The purchase order carries the agreed unit cost and Incoterms. When the vendor confirms the PO via EDI/portal, that cost is locked for that buy.
- Cost changes: Vendors file a Cost Change Reques

In [25]:
question = """
If a human buyer would negotiate any of these terms, where would be the biggest gain on your side and what would the vendor want the most?
"""

old_history = result
result = await agent.run(question, message_history=add_history(old_history))
print(result.output)

Biggest gains for us when a human leans in
- Unit cost and back-end funding: A 1–2% drop in ex‑vendor cost or an added growth rebate/MDF accrual goes straight to gross margin and is usually the single biggest lever. On long tail, back-end (tiered rebates, promo funding) is often easier to secure than headline cost cuts.
- Payment terms and cash flow: Moving from 30 to 60 days (or adding early‑pay discount options we control) materially improves working capital without touching retail.
- Exclusivity and assortment access: Early access, channel exclusives, or differentiated bundles drive conversion and defensibility; impact can exceed a small cost cut.
- Price protection and lifecycle support: Commitments on markdown support, RTV/capped returns, and defect allowances de‑risk end‑of‑life and reduce cost‑to‑serve.
- Landed cost/ops levers: Shifting to DDP, consolidating shipments, improving packaging to reduce damages, shorter lead times, lower MOQs, and higher fill rates—all shrink our tr

In [26]:
question = """
This is very interesting. If there were one point in the interaction with vendors (negotiations and otherwise) where AI would benefit you (for instance, where you currently do not negotiate or just use standard terms), what would that be? Which one area would have the biggest impact if AI was applied at scale?
"""

old_history = result
result = await agent.run(question, message_history=add_history(old_history))
print(result.output)

If I had to pick one, it’s programmatic, AI‑driven “deal building” for the long tail—automating the micro‑negotiations we currently skip and default to standard terms.

Why this has the biggest impact
- It hits the P&L directly at scale. Even 50–150 bps uplift on thousands of SKUs rolls up to real money, and we leave that on the table today because we don’t tailor terms vendor by vendor.
- Long‑tail vendors are heterogeneous. Some value cash speed, others traffic, others predictability. AI can match the right “carrots” to each vendor in a way a rules engine can’t.

What it would do
- Compose offers, not just counters: dynamically bundle unit cost, growth rebates/MDF, payment terms, freight/Incoterms, price protection windows, and merchandising slots into a single package per vendor/SKU.
- Predict acceptance and value: model each vendor’s likelihood to accept each lever, expected gross margin and cash impact by country (including FX, VAT/EPR, duties), and operational risk.
- Optimize un